In [1]:
from utils.dataset import read_dataset, generate_hold_out_split
data = read_dataset('data')
raw_training_data, raw_testing_data = generate_hold_out_split(data)
del data

In [2]:
from utils.preprocessing import preprocess_dataframe, extract_labels, decipher_labels
training_data = preprocess_dataframe(raw_training_data, 'training_data')
testing_data = preprocess_dataframe(raw_testing_data, 'testing_data')
training_labels = extract_labels(training_data)
testing_labels = extract_labels(testing_data)

In [3]:
from features import extract_features
training_features = extract_features(training_data, raw_training_data, 'training')
testing_features = extract_features(testing_data, raw_testing_data, 'testing')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.concat([training_data, testing_data])
body_ids = list(set(data['Body ID']))
train_body_ids, test_body_ids = train_test_split(body_ids, train_size=0.8)
test_set = data[data['Body ID'].isin(test_body_ids)]
test_headlines = list(set(test_set['Headline']))
train_set = data[data['Body ID'].isin(train_body_ids)]
train_set = train_set[~train_set['Headline'].isin(test_headlines)]
train_labels = extract_labels(train_set)
test_labels = extract_labels(test_set)

In [5]:
features = pd.concat([training_features, testing_features])
train_features = features.ix[train_set.index]
test_features = features.ix[test_set.index]

In [6]:
from features import flatten_features
flattened_train_features = flatten_features(train_features)
flattened_test_features = flatten_features(test_features)
from classifier import train_classifier
clf = train_classifier(flattened_train_features, train_labels)

/Users/eric/miniconda3/envs/fnc-1/lib/python3.5/site-packages/lightgbm-0.1-py3.5.egg/lightgbm/engine.py:163: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  booster = Booster(params=params, train_set=train_set)
/Users/eric/miniconda3/envs/fnc-1/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-04-14 17:21:44.601886. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [7]:
from utils.scoring import evaluate_submission
predictions = test_set.copy()
predictions['Stance'] = decipher_labels(clf.predict(flattened_test_features), index=test_features.index)
evaluate_submission(test_set, predictions)

/Users/eric/miniconda3/envs/fnc-1/lib/python3.5/site-packages/lightgbm-0.1-py3.5.egg/lightgbm/sklearn.py:598: UserWarning: Using Pandas (default) integer column names, not column indexes. You can use indexes with DataFrame.values.
  class_probs = self.predict_proba(X, raw_score, num_iteration)


CONFUSION MATRIX:
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    552    |     0     |    206    |    43     |
-------------------------------------------------------------
| disagree  |    92     |     0     |    31     |     3     |
-------------------------------------------------------------
|  discuss  |    695    |     0     |    833    |    157    |
-------------------------------------------------------------
| unrelated |    46     |     0     |    29     |   7467    |
-------------------------------------------------------------
ACCURACY: 0.872

MAX  - the best possible score (100% accuracy)
NULL - score as if all predicted stances were unrelated
TEST - score based on the provided predictions

||    MAX    ||    NULL   ||    TEST   ||
||  4497.5   ||  1885.5   ||  3507.75  ||

77.99332962757087% score achieved


/Users/eric/miniconda3/envs/fnc-1/lib/python3.5/json/encoder.py:198: DeprecationWarning: Interpreting naive datetime as local 2017-04-14 17:21:49.134630. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
